In [ ]:
import pygame
import sys
import random
import io


from google.colab import files
uploaded = files.upload()


# Define colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
BLUE = (0, 0, 255)
GREEN = (0, 255, 0)

# Define constants
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600
ROBOT_SIZE = 30
PATIENT_SIZE = 20
GRID_PADDING = 50
NEW_PATIENT_INTERVAL = 5  # Interval in seconds for new patient arrival

class NursingRobot:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.target_patient = None
        self.home_x = x
        self.home_y = y
        self.attended_patients = set()  # Set to store unique numbers of attended patients

    def move_to(self, x, y):
        dx = (x - self.x) / max(abs(x - self.x), 1) * 5
        dy = (y - self.y) / max(abs(y - self.y), 1) * 5
        self.x += dx
        self.y += dy

    def draw(self, screen):
        pygame.draw.circle(screen, RED, (int(self.x), int(self.y)), ROBOT_SIZE)

    def interact_with_patient(self, patient):
        distance = ((self.x - patient.x) ** 2 + (self.y - patient.y) ** 2) ** 0.5
        if distance < ROBOT_SIZE + PATIENT_SIZE:
            if patient.unique_number not in self.attended_patients:
                print("Taking vitals from patient with unique number {}".format( patient.unique_number))
                self.attended_patients.add(patient.unique_number)
                # Read patient vitals from file if ID matches
                with open("patient_vitals.txt", "r") as file:
                    for line in file:
                       patient_id, vitals = line.strip().split(",", maxsplit=1)
                       #print(patient_id)
                       if int(patient_id) == patient.unique_number:
                            print("\nVitals for patient ID {}: \nAge Sex BloodPressure SugarLevel Temperature LevelOfConsciousness OxygenSaturation bpm\n{}\n\n\n".format(patient_id, vitals))
                            with open("attended_patient_vitals.csv", "a") as attended_file:
                                #for vitals_data in attended_vitals:
                                attended_file.write("{},{}\n".format(patient_id, vitals))
            patient.attended = True
            self.target_patient = None  # Reset the target patient after interaction

    def return_home(self):
        self.move_to(self.home_x, self.home_y)

class Patient:
    patient_count = 0  # Class variable to keep track of patient count

    def __init__(self, x, y, grid_x, grid_y):
        self.x = x
        self.y = y
        self.grid_x = grid_x
        self.grid_y = grid_y
        self.attended = False
        Patient.patient_count += 1
        self.unique_number = Patient.patient_count  # Assigning unique number to the patient

    def draw(self, screen):
        color = GREEN if self.attended else BLUE
        pygame.draw.circle(screen, color, (int(self.x), int(self.y)), PATIENT_SIZE)

def add_new_patient(patients, robot):
    min_distance = ROBOT_SIZE + PATIENT_SIZE + 50  # Minimum distance between robot home and patient
    home_grid_x = robot.home_x // (SCREEN_WIDTH // 5)
    home_grid_y = robot.home_y // (SCREEN_HEIGHT // 5)
    while True:
        x = random.randint(GRID_PADDING, SCREEN_WIDTH - GRID_PADDING)
        y = random.randint(GRID_PADDING, SCREEN_HEIGHT - GRID_PADDING)
        grid_x = x // (SCREEN_WIDTH // 5)
        grid_y = y // (SCREEN_HEIGHT // 5)
        if (grid_x, grid_y) != (home_grid_x, home_grid_y):
            break
        # if grid_x == 0 and grid_y == 0:
        #   ward_number = 1
        # elif grid_x == 4 and grid_y == 0:
        #   ward_number = 2
        # elif grid_x == 0 and grid_y == 4:
        #   ward_number = 3
        # elif grid_x == 4 and grid_y == 4:
        #     ward_number = 4
        # else:
        #    ward_number = 5
    patients.append(Patient(x, y, grid_x, grid_y))
def main():
    pygame.init()

    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    pygame.display.set_caption("Nursing Robot Simulation")

    clock = pygame.time.Clock()

    robot = NursingRobot(SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2)

    # Calculate grid cell width and height
    cell_width = (SCREEN_WIDTH - 2 * GRID_PADDING) // 5
    cell_height = (SCREEN_HEIGHT - 2 * GRID_PADDING) // 5

    patients = []

    patient_index = 0  # Index of the patient the robot is currently interacting with
    new_patient_timer = pygame.time.get_ticks()  # Initialize the timer

    font = pygame.font.Font(None, 24)  # Font for grid numbers

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        # Move the robot towards the current patient
        if patients:  # Check if there are patients in the list
            current_patient = patients[patient_index]
            robot.move_to(current_patient.x, current_patient.y)

        screen.fill(WHITE)

        # Draw grid lines and numbers
        for i in range(6):
            # Draw horizontal grid lines
            pygame.draw.line(screen, BLACK, (GRID_PADDING, GRID_PADDING + i * cell_height), (SCREEN_WIDTH - GRID_PADDING, GRID_PADDING + i * cell_height))
            # Draw vertical grid lines
            pygame.draw.line(screen, BLACK, (GRID_PADDING + i * cell_width, GRID_PADDING), (GRID_PADDING + i * cell_width, SCREEN_HEIGHT - GRID_PADDING))
            # Display grid numbers
            for j in range(5):
                number = font.render(str(j + i * 5), True, BLACK)
                screen.blit(number, (GRID_PADDING + j * cell_width + cell_width // 2 - 5, GRID_PADDING + i * cell_height + cell_height // 2 - 10))

        # Draw and position patients
        for patient in patients:
            patient_x = GRID_PADDING + patient.grid_x * cell_width + cell_width // 2
            patient_y = GRID_PADDING + patient.grid_y * cell_height + cell_height // 2
            patient.x = patient_x
            patient.y = patient_y
            patient.draw(screen)
            robot.interact_with_patient(patient)  # Check interaction with each patient

        robot.draw(screen)
        pygame.display.flip()

        # Move to the next patient
        if patients:  # Check if there are patients in the list
            if current_patient.attended:
                patient_index = (patient_index + 1) % len(patients)

        # If all patients have been attended, return home
        if all(patient.attended for patient in patients):
            robot.return_home()


        if pygame.time.get_ticks() - new_patient_timer >= NEW_PATIENT_INTERVAL * 1000:
            add_new_patient(patients, robot)
            new_patient_timer = pygame.time.get_ticks()

        clock.tick(30)

if __name__ == "__main__":
    main()
